In [1]:
# Only use "ADNIMERGE.csv" as phenotype and covariate data.

import numpy as np
import pandas as pd

Load sample names of VCF data.

In [2]:
VCFsamplelist = pd.read_csv("ADNI/vcfsample_list.txt", header = None)
VCFsamplelist_IND = VCFsamplelist.applymap(lambda x: x.split("_")[0]).iloc[:,0]
VCFsamplelist_IND = VCFsamplelist_IND.rename('IND')
VCFsamplelist_ID = VCFsamplelist.applymap(lambda x: x.split("_")[1]).iloc[:,0]
VCFsamplelist_ID = VCFsamplelist_ID.rename('ID')
VCFsamplelist = pd.concat([VCFsamplelist_IND,VCFsamplelist_ID],axis=1)

In [3]:
VCFsamplelist = VCFsamplelist.rename(columns={'IND':'FID', 'ID':'IID'})
#VCFsamplelist_data.to_csv("keep_samples.txt", sep="\t", index=False)
VCFsamplelist = VCFsamplelist.sort_values('IID')
VCFsamplelist

,FID,IID
16,17,0002
581,587,0008
316,319,0015
616,622,0021
688,694,0023
...,...,...
668,674,4676
732,739,4686
483,488,4692
749,756,4696


In [4]:
# Merge summary data.
ADNI_BASIC_data = pd.read_csv("ADNI/ADNIMERGE.csv")
ADNI_BASIC_data = ADNI_BASIC_data[ADNI_BASIC_data.iloc[:,4]!="ADNI3"]
ADNI_BASIC_data = ADNI_BASIC_data[ADNI_BASIC_data["VISCODE"]=="bl"]

ADNI_BASIC_data.loc[ADNI_BASIC_data["ABETA_bl"]==">1700","ABETA_bl"] = 1700
ADNI_BASIC_data.loc[ADNI_BASIC_data["ABETA_bl"]=="<200","ABETA_bl"] =200
ADNI_BASIC_data = ADNI_BASIC_data.astype({"ABETA_bl": "float"})
ADNI_BASIC_data.loc[ADNI_BASIC_data["TAU_bl"]==">1300","TAU_bl"] = 1300
ADNI_BASIC_data.loc[ADNI_BASIC_data["TAU_bl"]=="<80","TAU_bl"] = 80
ADNI_BASIC_data = ADNI_BASIC_data.astype({"TAU_bl": "float"})
ADNI_BASIC_data.loc[ADNI_BASIC_data["PTAU_bl"]==">120","PTAU_bl"] = 120
ADNI_BASIC_data.loc[ADNI_BASIC_data["PTAU_bl"]=="<8","PTAU_bl"] = 8
ADNI_BASIC_data = ADNI_BASIC_data.astype({"PTAU_bl": "float"})
    
ADNI_BASIC_data = ADNI_BASIC_data[~np.isnan(ADNI_BASIC_data["ABETA_bl"])]
ADNI_BASIC_data = ADNI_BASIC_data[["RID", "VISCODE", "AGE", "PTGENDER", "APOE4", "ABETA_bl","TAU_bl","PTAU_bl", "EXAMDATE"]]

ADNI_BASIC_data = ADNI_BASIC_data.astype({"RID": "int64"})
ADNI_BASIC_data["RID"] = ADNI_BASIC_data["RID"].apply(lambda x:str(x).zfill(4))
ADNI_BASIC_data = ADNI_BASIC_data.sort_values("RID")

ADNI_BASIC_data = ADNI_BASIC_data[ADNI_BASIC_data["RID"].isin(VCFsamplelist["IID"])]

/Users/yuichiro/Workspace/ADabetaQTL/PaperCode/.env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3173: DtypeWarning: Columns (18,19,20,103,104) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
def gender2val(df):
    if df["PTGENDER"] == "Male":
        return 1
    elif df["PTGENDER"] == "Female":
        return 0

for rid in ADNI_BASIC_data["RID"].iteritems():
    ridval = rid[1]
    #print(ADNI_BASIC_data.loc[ADNI_BASIC_data["RID"]==rid[1],:])
    ADNI_BASIC_data.loc[ADNI_BASIC_data["RID"]==ridval,"FID"] = VCFsamplelist.loc[VCFsamplelist["IID"]==ridval,"FID"].values
ADNI_BASIC_data["SEX"] = ADNI_BASIC_data.apply(gender2val, axis=1)
ADNI_BASIC_data = ADNI_BASIC_data.rename(columns={'RID': 'IID'})
ADNI_BASIC_data = ADNI_BASIC_data[["FID","IID","ABETA_bl","TAU_bl","PTAU_bl","AGE","SEX","APOE4","EXAMDATE"]]
ADNI_BASIC_data = ADNI_BASIC_data.astype({"FID": "int64"})
ADNI_BASIC_data = ADNI_BASIC_data.sort_values("FID")


ADNI_BASIC_data_cov = ADNI_BASIC_data[["FID","IID","AGE","SEX","APOE4"]]
ADNI_BASIC_data_cov.to_csv("cov_merge_ADNI_CSF.txt", sep="\t", index=False)

ADNI_BASIC_data_pheno = ADNI_BASIC_data[["FID","IID","ABETA_bl","TAU_bl","PTAU_bl"]]
ADNI_BASIC_data_pheno.to_csv("pheno_merge_ADNI_CSF.txt", sep="\t", index=False)

In [6]:
ADNI_BASIC_data

,FID,IID,ABETA_bl,TAU_bl,PTAU_bl,AGE,SEX,APOE4,EXAMDATE
106,1,0056,1048.0,128.2,12.23,69.6,0,0.0,2005-12-22
122,3,0059,1605.0,186.4,16.69,70.9,0,0.0,2005-12-22
12148,5,2237,1700.0,185.3,16.42,65.0,1,0.0,2011-01-26
316,6,0123,1566.0,290.1,26.30,73.3,1,0.0,2006-02-07
563,8,0256,288.4,293.9,27.12,70.1,1,2.0,2006-04-05
...,...,...,...,...,...,...,...,...,...
2870,813,2042,827.6,124.1,10.58,69.5,1,0.0,2010-08-25
7802,815,4302,948.3,321.1,31.98,76.1,1,0.0,2011-11-18
2928,816,2109,532.1,274.7,28.79,75.2,1,0.0,2010-10-18
8336,817,4244,654.3,145.4,12.16,91.4,1,0.0,2011-11-22
